In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss


from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
from catboost import CatBoostClassifier

In [2]:
from scipy.sparse import csr_matrix
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr


In [4]:
train_csr = load_csr('../../data/features/xgb/train_csr_2500')
predict_csr = load_csr('../../data/features/xgb/predict_csr_2500')
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')

data load
indices load
indptr load
data load
indices load
indptr load


In [5]:
print(predict_csr.shape)
print(train_csr.shape)

(80276, 2363)
(2992639, 2363)


In [6]:

predict_result = predicted_file[['instance_id']].copy()
predict_result['predicted_score'] = 0


# Define catboost

In [7]:
# lgb_model = lgb.LGBMClassifier(
#     boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
#     max_depth=-1, n_estimators=5000, objective='binary',
#     subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
#     learning_rate=0.05, random_state=1001, n_jobs=16
# )


lgb_model = CatBoostClassifier(iterations=60000, 
                               use_best_model=True,
                              depth=None, 
                              reg_lambda = 0.1,
                              learning_rate=0.05, 
                              loss_function='Logloss',
                               eval_metric='Logloss',
                              verbose=500,
                               early_stopping_rounds=2,
                              task_type='GPU')


In [ ]:

import gc
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, random_state=21, shuffle=True)
best_score = []
gc.collect()
oof = np.zeros(len(train_y))
predict_x = predict_csr.toarray()
predict_x = predict_x.astype('int')
train_csr_dense = train_csr.toarray()
train_csr_dense = train_csr_dense.astype('int')


cate_index = np.array([i for i in range(train_csr_dense.shape[1])])


        
print('dense converted done!')
for index, (train_index, test_index) in enumerate(skf.split(train_csr_dense, train_y)):
    lgb_model.fit(train_csr_dense[train_index], train_y[train_index],
#                   cat_features=cate_index,
                  eval_set=[
                            (train_csr_dense[test_index], train_y[test_index])], early_stopping_rounds=2,
                  verbose_eval=500,metric_period=500)

    val_preds = lgb_model.predict_proba(train_csr_dense[test_index], ntree_end=lgb_model.tree_count_)[:, 1]
    score = log_loss(train_y[test_index],val_preds)
    best_score.append(score)
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_x, ntree_end=lgb_model.tree_count_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    oof[test_index] = val_preds
    gc.collect()

predict_result['predicted_score'] = predict_result['predicted_score']/n_fold

dense converted done!


In [8]:
cv_loss = np.mean(best_score)
colnum = 1353
now = str(np.random.randint(1000000))
predict_result[['instance_id', 'predicted_score']].to_csv( "submission/%s_lgb_n%d_b%d_1h_col%d.csv" % (now, n_fold, 70, colnum), index=False)
np.save('submission/oof_%s_%.5f'%(now, cv_loss), oof) 

# grid search

In [5]:
import gc
predict_x = predict_csr.toarray()
predict_x = predict_x.astype('int')
train_csr_dense = train_csr.toarray()
train_csr_dense = train_csr_dense.astype('int')

print('start...')
lr_list=[0.05,0.1,0.03]
reg_lambda_list = [0.1,0.2,0.05]
depth_list = [None,6,8]
for lr in lr_list:
    for rl in reg_lambda_list:
        for d in depth_list:
            lgb_model = CatBoostClassifier(iterations=60000, 
                               use_best_model=True,
                              depth=d, 
                              reg_lambda = rl,
                              learning_rate=lr, 
                              loss_function='Logloss',
                               eval_metric='Logloss',
                              verbose=500,
                               early_stopping_rounds=2,
                              task_type='GPU')
            
            
            predict_result = predicted_file[['instance_id']].copy()
            predict_result['predicted_score'] = 0

            n_fold = 5
            skf = StratifiedKFold(n_splits=n_fold, shuffle=True)
            best_score = []
            gc.collect()
            oof = np.zeros(len(train_y))

            print('dense converted done!')
            for index, (train_index, test_index) in enumerate(skf.split(train_csr_dense, train_y)):
                lgb_model.fit(train_csr_dense[train_index], train_y[train_index],
            #                   cat_features=cate_index,
                              eval_set=[
                                        (train_csr_dense[test_index], train_y[test_index])], early_stopping_rounds=2,
                              verbose_eval=500,metric_period=500)

                val_preds = lgb_model.predict_proba(train_csr_dense[test_index], ntree_end=lgb_model.tree_count_)[:, 1]
                score = log_loss(train_y[test_index],val_preds)
                best_score.append(score)
                print(best_score)
                print('cv:',np.mean(best_score))
                test_pred = lgb_model.predict_proba(predict_x, ntree_end=lgb_model.tree_count_)[:, 1]
                print('test mean:', test_pred.mean())
                predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
                oof[test_index] = val_preds
                gc.collect()

            predict_result['predicted_score'] = predict_result['predicted_score']/n_fold
            
            cv_loss = np.mean(best_score)
            colnum = 2387
            now = str(np.random.randint(1000000))
            predict_result[['instance_id', 'predicted_score']].to_csv( "submission/%s_catboost_n%d_b%d_1h_col%d.csv" % (now, n_fold, 70, colnum), index=False)
            np.save('submission/oof_%s_%.5f'%(now, cv_loss), oof) 

start...
dense converted done!
0:	learn: 0.6584996	test: 0.6585220	best: 0.6585220 (0)	total: 85.3ms	remaining: 1h 25m 20s
500:	learn: 0.4183454	test: 0.4190539	best: 0.4190539 (500)	total: 25.4s	remaining: 50m 21s
1000:	learn: 0.4162262	test: 0.4175245	best: 0.4175245 (1000)	total: 49.4s	remaining: 48m 30s
1500:	learn: 0.4148576	test: 0.4168329	best: 0.4168329 (1500)	total: 1m 14s	remaining: 48m 23s
2000:	learn: 0.4137482	test: 0.4164212	best: 0.4164212 (2000)	total: 1m 40s	remaining: 48m 25s
2500:	learn: 0.4127527	test: 0.4161187	best: 0.4161187 (2500)	total: 2m 6s	remaining: 48m 22s
3000:	learn: 0.4118252	test: 0.4159223	best: 0.4159223 (3000)	total: 2m 32s	remaining: 48m 15s
3500:	learn: 0.4109712	test: 0.4157767	best: 0.4157767 (3500)	total: 2m 58s	remaining: 48m 3s
4000:	learn: 0.4101110	test: 0.4156740	best: 0.4156740 (4000)	total: 3m 25s	remaining: 47m 55s
4500:	learn: 0.4092939	test: 0.4155766	best: 0.4155766 (4500)	total: 3m 51s	remaining: 47m 39s
5000:	learn: 0.4085035	test:

[0.4153044580660038, 0.41486060006001124, 0.41494589028093937, 0.4153603321384938]
cv: 0.41511782013636206
test mean: 0.21716140793170816
0:	learn: 0.6597236	test: 0.6596689	best: 0.6596689 (0)	total: 75.3ms	remaining: 1h 15m 19s
500:	learn: 0.4185315	test: 0.4181572	best: 0.4181572 (500)	total: 24s	remaining: 47m 34s
1000:	learn: 0.4164200	test: 0.4166148	best: 0.4166148 (1000)	total: 46.5s	remaining: 45m 42s
1500:	learn: 0.4150891	test: 0.4159124	best: 0.4159124 (1500)	total: 1m 10s	remaining: 45m 43s
2000:	learn: 0.4139912	test: 0.4154936	best: 0.4154936 (2000)	total: 1m 34s	remaining: 45m 33s
2500:	learn: 0.4129797	test: 0.4151854	best: 0.4151854 (2500)	total: 1m 58s	remaining: 45m 27s
3000:	learn: 0.4120591	test: 0.4149818	best: 0.4149818 (3000)	total: 2m 23s	remaining: 45m 25s
3500:	learn: 0.4111913	test: 0.4148473	best: 0.4148473 (3500)	total: 2m 48s	remaining: 45m 17s
4000:	learn: 0.4103408	test: 0.4147335	best: 0.4147335 (4000)	total: 3m 13s	remaining: 45m 8s
4500:	learn: 0.40

7500:	learn: 0.4048341	test: 0.4150421	best: 0.4150421 (7500)	total: 6m 10s	remaining: 43m 14s
8000:	learn: 0.4041102	test: 0.4150419	best: 0.4150419 (8000)	total: 6m 36s	remaining: 42m 55s
8500:	learn: 0.4033950	test: 0.4150316	best: 0.4150316 (8500)	total: 7m 1s	remaining: 42m 36s
9000:	learn: 0.4026800	test: 0.4150225	best: 0.4150225 (9000)	total: 7m 27s	remaining: 42m 16s
9500:	learn: 0.4019682	test: 0.4150365	best: 0.4150225 (9000)	total: 7m 52s	remaining: 41m 53s
10000:	learn: 0.4012859	test: 0.4150664	best: 0.4150225 (9000)	total: 8m 18s	remaining: 41m 31s
bestTest = 0.4150225
bestIteration = 9000
[0.4147628238460549, 0.41498741438816295, 0.4150225233377266]
cv: 0.4149242538573148
test mean: 0.21677460664110476
0:	learn: 0.6586052	test: 0.6586295	best: 0.6586295 (0)	total: 91.6ms	remaining: 1h 31m 33s
500:	learn: 0.4183106	test: 0.4187731	best: 0.4187731 (500)	total: 24s	remaining: 47m 28s
1000:	learn: 0.4162409	test: 0.4172381	best: 0.4172381 (1000)	total: 46.5s	remaining: 45m 

0:	learn: 0.6588383	test: 0.6588663	best: 0.6588663 (0)	total: 90.4ms	remaining: 1h 30m 24s
500:	learn: 0.4164902	test: 0.4180017	best: 0.4180017 (500)	total: 33s	remaining: 1h 5m 19s
1000:	learn: 0.4136340	test: 0.4169219	best: 0.4169219 (1000)	total: 1m 6s	remaining: 1h 4m 52s
1500:	learn: 0.4112540	test: 0.4164191	best: 0.4164191 (1500)	total: 1m 40s	remaining: 1h 5m 16s
2000:	learn: 0.4090224	test: 0.4161654	best: 0.4161654 (2000)	total: 2m 15s	remaining: 1h 5m 19s
2500:	learn: 0.4068645	test: 0.4160183	best: 0.4160183 (2500)	total: 2m 50s	remaining: 1h 5m 31s
3000:	learn: 0.4047549	test: 0.4159514	best: 0.4159514 (3000)	total: 3m 26s	remaining: 1h 5m 27s
3500:	learn: 0.4027063	test: 0.4158928	best: 0.4158928 (3500)	total: 4m 3s	remaining: 1h 5m 21s
4000:	learn: 0.4006840	test: 0.4158663	best: 0.4158663 (4000)	total: 4m 39s	remaining: 1h 5m 14s
4500:	learn: 0.3986767	test: 0.4158387	best: 0.4158387 (4500)	total: 5m 16s	remaining: 1h 4m 59s
5000:	learn: 0.3967222	test: 0.4158331	bes

2000:	learn: 0.4138214	test: 0.4161808	best: 0.4161808 (2000)	total: 1m 34s	remaining: 45m 45s
2500:	learn: 0.4128604	test: 0.4159100	best: 0.4159100 (2500)	total: 1m 59s	remaining: 45m 39s
3000:	learn: 0.4119956	test: 0.4157230	best: 0.4157230 (3000)	total: 2m 23s	remaining: 45m 33s
3500:	learn: 0.4111454	test: 0.4155738	best: 0.4155738 (3500)	total: 2m 48s	remaining: 45m 25s
4000:	learn: 0.4103188	test: 0.4154134	best: 0.4154134 (4000)	total: 3m 13s	remaining: 45m 13s
4500:	learn: 0.4095073	test: 0.4153053	best: 0.4153053 (4500)	total: 3m 39s	remaining: 45m 2s
5000:	learn: 0.4087338	test: 0.4152216	best: 0.4152216 (5000)	total: 4m 4s	remaining: 44m 46s
5500:	learn: 0.4079649	test: 0.4151648	best: 0.4151648 (5500)	total: 4m 29s	remaining: 44m 29s
6000:	learn: 0.4072271	test: 0.4151175	best: 0.4151175 (6000)	total: 4m 54s	remaining: 44m 12s
6500:	learn: 0.4064835	test: 0.4150732	best: 0.4150732 (6500)	total: 5m 20s	remaining: 43m 53s
7000:	learn: 0.4057576	test: 0.4150328	best: 0.41503

8500:	learn: 0.4035769	test: 0.4150853	best: 0.4150853 (8500)	total: 7m 6s	remaining: 43m 2s
9000:	learn: 0.4028728	test: 0.4150758	best: 0.4150758 (9000)	total: 7m 32s	remaining: 42m 42s
9500:	learn: 0.4022039	test: 0.4150910	best: 0.4150758 (9000)	total: 7m 58s	remaining: 42m 24s
10000:	learn: 0.4015143	test: 0.4150944	best: 0.4150758 (9000)	total: 8m 24s	remaining: 42m 2s
bestTest = 0.4150757524
bestIteration = 9000
[0.41507597895904486]
cv: 0.41507597895904486
test mean: 0.21555186268013793
0:	learn: 0.6586088	test: 0.6585600	best: 0.6585600 (0)	total: 81.2ms	remaining: 1h 21m 12s
500:	learn: 0.4183388	test: 0.4188659	best: 0.4188659 (500)	total: 24.4s	remaining: 48m 17s
1000:	learn: 0.4162107	test: 0.4172778	best: 0.4172778 (1000)	total: 47.6s	remaining: 46m 48s
1500:	learn: 0.4149017	test: 0.4166245	best: 0.4166245 (1500)	total: 1m 11s	remaining: 46m 27s
2000:	learn: 0.4138188	test: 0.4161920	best: 0.4161920 (2000)	total: 1m 35s	remaining: 46m 14s
2500:	learn: 0.4128655	test: 0.4

6500:	learn: 0.4064433	test: 0.4150861	best: 0.4150861 (6500)	total: 5m 19s	remaining: 43m 51s
7000:	learn: 0.4057128	test: 0.4150628	best: 0.4150628 (7000)	total: 5m 45s	remaining: 43m 32s
7500:	learn: 0.4049980	test: 0.4150464	best: 0.4150464 (7500)	total: 6m 10s	remaining: 43m 13s
8000:	learn: 0.4043060	test: 0.4150405	best: 0.4150405 (8000)	total: 6m 36s	remaining: 42m 54s
8500:	learn: 0.4035858	test: 0.4150337	best: 0.4150337 (8500)	total: 7m 1s	remaining: 42m 35s
9000:	learn: 0.4028897	test: 0.4150415	best: 0.4150337 (8500)	total: 7m 27s	remaining: 42m 16s
9500:	learn: 0.4022125	test: 0.4150336	best: 0.4150336 (9500)	total: 7m 53s	remaining: 41m 56s
10000:	learn: 0.4015381	test: 0.4150440	best: 0.4150336 (9500)	total: 8m 19s	remaining: 41m 35s
10500:	learn: 0.4008626	test: 0.4150612	best: 0.4150336 (9500)	total: 8m 45s	remaining: 41m 15s
bestTest = 0.415033595
bestIteration = 9500
[0.41507597895904486, 0.4150240800037914, 0.41422062488989214, 0.4152435670839138, 0.415033340571275

KeyboardInterrupt: 

In [47]:
# import gc
# n_fold = 5
# skf = StratifiedKFold(n_splits=n_fold, random_state=19, shuffle=True)
# best_score = []
# gc.collect()
# oof = np.zeros(len(train_y))
# for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
#     lgb_model.fit(train_csr[train_index].todense(), train_y[train_index],
#                   eval_set=[
# #                               (train_csr_f[train_index].todense(), train_y[train_index]),
#                             (train_csr[test_index].todense(), train_y[test_index])], early_stopping_rounds=1,
#                   verbose_eval=500,metric_period=500)
#     break
#     best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
#     print(best_score)
#     print('cv:',np.mean(best_score))
#     test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
#     print('test mean:', test_pred.mean())
#     predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
#     oof[test_index] = lgb_model.predict_proba(train_csr_f[test_index], ntree_limit=lgb_model.best_iteration)[:, 1]
#     gc.collect()

# predict_result['predicted_score'] = predict_result['predicted_score']/5

In [48]:
# train_csr_f = train_csr[:100000]
# train_y_f = train_y[:100000]

# import gc
# n_fold = 5
# skf = StratifiedKFold(n_splits=n_fold, random_state=19, shuffle=True)
# best_score = []
# gc.collect()
# oof = np.zeros(len(train_y_f))
# for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
#     val_x = train_csr_f[test_index].todense()
#     lgb_model.fit(train_csr_f[train_index].todense(), train_y_f[train_index],
#                   eval_set=[
#                             (val_x, train_y_f[test_index])], early_stopping_rounds=1,
#                   verbose_eval=500,metric_period=500)

#     val_preds = lgb_model.predict_proba(val_x, ntree_end=lgb_model.tree_count_)[:, 1]
#     score = log_loss(train_y_f[test_index],val_preds)
#     best_score.append(score)
#     print(best_score)
#     print('cv:',np.mean(best_score))
#     test_pred = lgb_model.predict_proba(predict_csr.todense(), ntree_end=lgb_model.tree_count_)[:, 1]
#     print('test mean:', test_pred.mean())
#     predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
#     oof[test_index] = val_preds
#     gc.collect()

# predict_result['predicted_score'] = predict_result['predicted_score']/n_fold